In [1]:
from pykrx import stock

from typing import Dict, Optional
import pandas as pd
from pypfopt import EfficientFrontier

from data.data_loader import PykrxDataLoader
from simulation.account import Account
from simulation.broker import Broker
from simulation.utility import ticker_to_name, get_lookback_fromdate
from simulation.utility import rebalance
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio
from simulation.visualize import (plot_cumulative_return, plot_single_period_return,
                                  plot_relative_single_period_return,
                                  plot_cumulative_asset_profit, plot_asset_weight)

In [2]:
# # 특정 거래일 기준으로 전종목 주가 조회하기 (market = 의 기본값은 코스피)
# df = stock.get_market_ohlcv("20250220", market = "KOSDAQ")
# print(df)

# # 지정된 기간동안 조회
# # stock.get_market_ohlcv(조회시작일자(YYYYMMDD), 조회종료일자(YYYYMMDD),조회할종목코드, 데이터 빈도(D/M/Y)(기본값 D), 수정주가로표현할지(adjusted = True/False)(기본값 True), 거래소시장(KOSPI/KOSDAQ/ALL)(기본값 KOSPI))

# # 예시 : 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 반영된 일별 주가 데이터 출력 
# df = stock.get_market_ohlcv("20240101", "20241231", "005930")
# # 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 반영된 "월별" 주가 데이터 출력
# df = stock.get_market_ohlcv("20240101", "20241231", "005930", "m") 
# # 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 "반영되지 않은" 일별 주가 데이터 출력
# df = stock.get_market_ohlcv("20240101", "20241231", "005930", adjusted=False) 
# print(df)



# # 여러종목의 주가를 조회 가능하지만 대량의 요청은 서버 과부화의 위험떄문에 지연시간을 두어 실행함 
# import time
# ticker_list = ['005930', '000020', '035720']
# for ticker in ticker_list:
#     df = stock.get_market_ohlcv_by_date("20250101", "20250107", ticker)
#     print(df.head())
#     time.sleep(1) # 1초간 지연시키기

## 평균-분산 전략 구현

### 평균-분산 최적화

In [3]:
#자산별 수익률 계산[calculate_return(데이터프레임 이름)]
def calculate_return(ohlcv_data: pd.DataFrame):
    # 주가 데이터에서 날짜 인덱스와 종목코드, 종가 2개의 열을 선택하고 종목코드(ticker)와 날짜(date)를 인덱스 종가(close)를 열로하는 종가 데이터(close_data)로 변환
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date'])
    # unstack() 함수를 통해 종목코드(ticker)를 열로 변환 [날짜(date)가 인덱스, 종목 코드(ticker)를 열, 종가(close)를 값으로 하는 종가데이터로 변환]
    close_data = close_data.unstack(level=0)
    close_data = close_data['close']
    # 종목별로 동가데이터에 대해 수익률을 계산하고 100을 곱해 백분율로 변환
    return_data = close_data.pct_change(1) * 100
    return return_data

def get_mean_variance_weights(return_data: pd.DataFrame,
                              risk_aversion: int) -> Optional[Dict]:
    #수익률 데이터를 이용해서 과거 평균수익률과 표본 공분산 행렬을 계산한다.
    #수익률 계산
    expected_return = return_data.mean(skipna=False).to_list()
    #공분산 행렬 계산
    cov = return_data.cov(min_periods=len(return_data))
    #공분산 행렬에 누락값이 있다면 None을 반환
    if cov.isnull().values.any() or cov.empty:
        return None
    
    # 과거 평균 수익률을 기대 수익률로, 표본 공분산 행렬을 공분산 행렬로 지정하고 최적화 솔버를 OSQP로 설정해 EfficientFrontier 객체를 초기화 한다.
    #평균-분산 최적화
    ef = EfficientFrontier(
        expected_returns=expected_return,
        cov_matrix=cov,
        solver='OSQP'
    )
    #EfficientFrontier 객체의 max_quadratic_utility() 메서드를 사용해 평균-분산 모델의 최적화 식을 나타내는 유틸리티 목적함수를 최대화해 자산 편입 비중을 구한다.
    ef.max_quadratic_utility(risk_aversion=risk_aversion)
    #0에 가까운 편입 비중은 0으로 처리하고, 편입 비중 소수점은 반올림 하지 않음
    weights = dict(ef.clean_weights(rounding=None))
    return weights

### 평균-분산 시뮬레이션

In [4]:
# 시뮬레이션 프로그램

def simulate_mean_variance(ohlcv_data: pd.DataFrame, look_back: int):
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 수익률 계산
    return_data = calculate_return(ohlcv_data=ohlcv_data)

    for date, ohlcv in ohlcv_data.groupby('date'):
        print(date.date())

        # 주문 처리 및 거래 생성
        transactions = broker.process_order(dt=date, data=ohlcv, orders=account.orders)
        # 계좌 내 자산 포지션, 투자 포트폴리오, 주문 업데이트
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # 현재 날짜의 수익률 데이터
        return_data_slice = return_data.loc[:date].iloc[-look_back:]
        # 자산 편입 비중 계산 및 뒤처리
        weights = get_mean_variance_weights(return_data=return_data_slice,
                                            risk_aversion=3)
        rounded_weights = (None if weights is None else
                           {k: round(v, 3) for k, v in weights.items()})
        print(f'portfolio: {rounded_weights}')
        if weights is None:
            continue

        # 투자 포트폴리오 조정
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    return account

In [5]:
# # 사용 예시
# # 삼성전자, 동화약품, 카카오 세 종목에 대해 2024년 1월 1일부터 2024년 12월 31일까지 월간 데이터를 입력으로 주면 세 종목의 수익률 데이터가 나온다.
# calculate_return_data = calculate_return(ohlcv_data)

# print(calculate_return_data)

# # 예시의 세종목의 월별 데이터에서 누락값을 제거하고 평균-분산 최적화를 통해 편입 비중을 계산
# weight = get_mean_variance_weights(calculate_return_data.dropna(), risk_aversion=3.07)
# print(weight)

# #최적화 결과에 따른 편입 비중은 각각 0%, 76.7%, 23.3%로 나타난다.
# #편입 비중이 0%인 삼성전자는 포트폴리오에서 제외되고, 동화약품과 카카오는 각각 76.7%, 23.3%의 편입 비중을 가진다.

In [6]:
fromdate = '2020-07-10'
todate = '2023-09-27'
ticker_list =  ['005930', '000660', '207940', '051910',
                '006400', '005380', '000270', '005490', '035420']

total_look_back = 1 + 24
adj_fromdate = get_lookback_fromdate(fromdate=fromdate,
                                     lookback=total_look_back, freq='m')

data_loader = PykrxDataLoader(fromdate=adj_fromdate,
                              todate=todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list,
                                         freq='m', delay=1)
ohlcv_data.head(15)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)
/Users/kimboyun/Desktop/Code/TeamProject/data/data_loader.py:116: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(rule).reset_index(level=0)


,ticker,open,high,low,close,volume
date,,,,,,
2018-07-31,000270,31800,31950,31500,31700,636196
2018-08-31,000270,31650,32600,30950,32050,13377137
2018-09-30,000270,31900,35600,31350,35100,19573161
2018-10-31,000270,35150,35200,26200,28400,27693336
2018-11-30,000270,28300,30750,27100,30500,19094252
2018-12-31,000270,30900,34200,29600,33700,16249408
2019-01-31,000270,33450,36650,31800,36350,21942579
2019-02-28,000270,36350,37350,34750,36550,15087190
2019-03-31,000270,36500,36500,33350,35350,18778392


In [10]:
fromdate = '2024-01-01'
todate = '2024-12-31'
total_look_back = 1 + 24
adj_fromdate = get_lookback_fromdate(fromdate=fromdate,
                                     lookback=total_look_back, freq='m')
data_loader = PykrxDataLoader(fromdate = adj_fromdate,
                              todate=todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list,
                                         freq='m', delay=1)
simulation_account = simulate_mean_variance(ohlcv_data=ohlcv_data, look_back=24)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)


2022-01-31
portfolio: None
2022-02-28
portfolio: None
2022-03-31
portfolio: None
2022-04-30
portfolio: None
2022-05-31
portfolio: None
2022-06-30
portfolio: None
2022-07-31
portfolio: None
2022-08-31
portfolio: None
2022-09-30
portfolio: None
2022-10-31
portfolio: None
2022-11-30
portfolio: None
2022-12-31
portfolio: None
2023-01-31
portfolio: None
2023-02-28
portfolio: None
2023-03-31
portfolio: None
2023-04-30
portfolio: None
2023-05-31
portfolio: None
2023-06-30
portfolio: None
2023-07-31
portfolio: None
2023-08-31
portfolio: None
2023-09-30
portfolio: None
2023-10-31
portfolio: None
2023-11-30
portfolio: None
2023-12-31
portfolio: None
2024-01-31
portfolio: {'000270': 0.0, '000660': 0.0, '005380': 0.378, '005490': 0.0, '005930': 0.0, '006400': 0.0, '035420': 0.0, '051910': 0.0, '207940': 0.622}
2024-02-29
portfolio: {'000270': 0.0, '000660': 0.0, '005380': 0.309, '005490': 0.0, '005930': 0.0, '006400': 0.0, '035420': 0.0, '051910': 0.0, '207940': 0.691}
2024-03-31
portfolio: {'0002

/Users/kimboyun/Desktop/Code/TeamProject/data/data_loader.py:116: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(rule).reset_index(level=0)


In [8]:
# 시뮬레이션 분석
# 시뮬레이션 결과 전처리

In [9]:
# 히스토리 형식 변환

df_account = pd.DataFrame(simulation_account.account_history).set_index('date')
df_portfolio = pd.DataFrame(simulation_account.portfolio_history).set_index('date')
df_portfolio = df_portfolio.assign(
    ticker=df_portfolio['ticker'].apply(lambda x: f'{x}({ticker_to_name(x)})')
)